## Import tools

In [1]:
import os
from distutils.dir_util import copy_tree
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import preprocessing
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Dense, Dropout, Flatten, experimental

In [2]:
# copy data current directory to working directory
main_dir = "../input/microsoft-catsvsdogs-dataset/PetImages/"

# create dataset directory path
work_dir = "/kaggle/working/PetImages/"

copy_tree(main_dir, work_dir)

['/kaggle/working/PetImages/Dog/7981.jpg',
 '/kaggle/working/PetImages/Dog/6234.jpg',
 '/kaggle/working/PetImages/Dog/1269.jpg',
 '/kaggle/working/PetImages/Dog/3863.jpg',
 '/kaggle/working/PetImages/Dog/6241.jpg',
 '/kaggle/working/PetImages/Dog/10304.jpg',
 '/kaggle/working/PetImages/Dog/623.jpg',
 '/kaggle/working/PetImages/Dog/2193.jpg',
 '/kaggle/working/PetImages/Dog/11925.jpg',
 '/kaggle/working/PetImages/Dog/3750.jpg',
 '/kaggle/working/PetImages/Dog/11378.jpg',
 '/kaggle/working/PetImages/Dog/2008.jpg',
 '/kaggle/working/PetImages/Dog/10730.jpg',
 '/kaggle/working/PetImages/Dog/5982.jpg',
 '/kaggle/working/PetImages/Dog/7737.jpg',
 '/kaggle/working/PetImages/Dog/2081.jpg',
 '/kaggle/working/PetImages/Dog/10597.jpg',
 '/kaggle/working/PetImages/Dog/6588.jpg',
 '/kaggle/working/PetImages/Dog/10054.jpg',
 '/kaggle/working/PetImages/Dog/7966.jpg',
 '/kaggle/working/PetImages/Dog/3919.jpg',
 '/kaggle/working/PetImages/Dog/6197.jpg',
 '/kaggle/working/PetImages/Dog/10924.jpg',
 '/ka

In [3]:
# Let's create function to filter & remove corrupted images
def CorruptImage():
    num_corrupted = 0
    for category in ("Cat", "Dog"):
        path = os.path.join(work_dir, category)

        for img in os.listdir(path):
            image_path = os.path.join(path, img)
            try:
                file_Obj = open(image_path, "rb")
                is_jfif = tf.compat.as_bytes("JFIF") in file_Obj.peek(10)
            finally:
                file_Obj.close()
            
            if not is_jfif:
                num_corrupted +=1
                os.remove(image_path)
    print("Total deleted images is = %d" % num_corrupted)

CorruptImage()

Total deleted images is = 1590


In [4]:
# Let's function to remove non jpg images
def NonJpg():
    j=0
    for catagory in os.listdir(work_dir):
        sub_folder = os.path.join(work_dir, catagory)
        for name in os.listdir(sub_folder):
            img_path = os.path.join(sub_folder,name)
            if name.split('.')[1] == 'jpg':
                if os.path.getsize(img_path) <= 0:
                    if os.path.isfile(img_path) == False:
                        print(img_path)
                    os.remove(img_path)
                elif os.path.getsize(img_path) > 0:
                    j +=1
            if name.split('.')[1] != 'jpg':
                print(img_path)
                os.remove(img_path)
    print(j)
NonJpg()

/kaggle/working/PetImages/Cat/Thumbs.db
/kaggle/working/PetImages/Dog/Thumbs.db
23410


In [5]:
# Let's split image dataset into Train & validation

image_size = (128, 128)

train_dataset = preprocessing.image_dataset_from_directory(
    work_dir,
    image_size = image_size,
    validation_split= 0.2,
    subset="training",
    batch_size =32,    
    seed = 1337,
    shuffle = True,
)

test_dataset = preprocessing.image_dataset_from_directory(
    work_dir,
    image_size = image_size,
    validation_split = 0.2,
    subset = "validation",
    batch_size = 32,    
    seed = 1337,
    shuffle= True,
)

Found 23410 files belonging to 2 classes.
Using 18728 files for training.



User settings:

   KMP_AFFINITY=granularity=fine,noverbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_SETTINGS=1
   KMP_WARNINGS=0

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=false
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_PLAIN_BARRIER='2,2'
   KMP_PLAIN_BARRIER_PATTERN='hyper,hyper'
   KMP_REDUCTION_BARRIER='1,1'


Found 23410 files belonging to 2 classes.
Using 4682 files for validation.


In [6]:
input_shapes=(128, 128, 3)

model=Sequential()

model.add(experimental.preprocessing.Rescaling(1.0/255, input_shape=(input_shapes)))

model.add(Conv2D(16,(3,3),activation='relu',input_shape=input_shapes))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(32,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

In [7]:
model.compile(
    loss='binary_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)

model.fit(
    train_dataset, 
    validation_data=test_dataset,
    epochs=15
)

Epoch 1/15


2022-02-08 17:18:20.585242: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
Corrupt JPEG data: 2226 extraneous bytes before marker 0xd9


326/586 [===============>..............] - ETA: 44s - loss: 0.7072 - accuracy: 0.6385

Corrupt JPEG data: 228 extraneous bytes before marker 0xd9


357/586 [=================>............] - ETA: 39s - loss: 0.6983 - accuracy: 0.6425

...

463/586 [======================>.......] - ETA: 20s - loss: 0.2139 - accuracy: 0.9110

Corrupt JPEG data: 239 extraneous bytes before marker 0xd9


586/586 [==============================] - ETA: 0s - loss: 0.2125 - accuracy: 0.9119

Corrupt JPEG data: 252 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1153 extraneous bytes before marker 0xd9
Corrupt JPEG data: 162 extraneous bytes before marker 0xd9
Corrupt JPEG data: 214 extraneous bytes before marker 0xd9
Corrupt JPEG data: 99 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1403 extraneous bytes before marker 0xd9


586/586 [==============================] - 106s 181ms/step - loss: 0.2125 - accuracy: 0.9119 - val_loss: 0.4609 - val_accuracy: 0.8167


In [8]:
# Let's save the trained model
keras.models.save_model(
    model,
    "cats_dogs_classification_model.h5"
)

#### Let's download some images from Google and check how does it work!!!

In [9]:
from keras.preprocessing import image
for i in range(1,8):
    test_image = image.load_img('../input/testimages/test' + str(i) + ".jpg", target_size = (128, 128))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = model.predict(test_image)
    if result[0][0]*100 > 50:
        prediction = 'Dog'
    elif result[0][0]*100 < 50:
        prediction = 'Cat'
    else:
        prediction = 'Can not recognize!'
    print("Your input : " + prediction)

NameError: name 'np' is not defined